In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import bisect
import math
import os
import time
from collections import deque

In [ ]:
""" LATTICE-BASED DBSCAN """
class DBSCAN:
    def __init__(self, df, config):
        self.df = df
        self.numRow = config['NumRow']
        self.numCol = config['NumCol']
        self.min_x = config['Min_x']
        self.min_y = config['Min_y']
        self.increment = config['Increment']
        self.vesselType = config['VesselType']
        self.minSC = config['MinSC']
        self.minNeigb = config['MinNeigb']
        
        self.lattice = self.initialize_lattice()
    
    def get_index(self, pos_x, pos_y):
        index_x = math.ceil((pos_x - self.min_x) / self.increment) - 1
        index_y = self.numRow - math.ceil((pos_y - self.min_y) / self.increment)
        if pos_x == self.min_x:
            index_x = 0
        if pos_y == self.min_y:
            index_y = self.numRow-1
        return index_x, index_y
    
    def initialize_lattice(self):
        lattice = [[None] * self.numCol for _ in range(self.numRow)]
        for i in range(self.numRow):
            for j in range(self.numCol):
                lattice[self.numRow-1-i][j] = {'x': self.min_x + j*self.increment,
                                               'y': self.min_y + i*self.increment,
                                               'code': None,
                                               'sampleCount': 0,
                                               'type/vslid': self.vesselType,
                                               'label': None,
                                               'clusterId': None}
        for idx, row in self.df.iterrows():
            index_x, index_y = self.get_index(row['x'], row['y'])
            lattice[index_y][index_x]['sampleCount'] += 1
        return lattice
    
    def get_neighbors(self, col, row):
        positions = [(col-1, row), (col-1, row-1), 
                     (col, row-1), (col+1, row-1), 
                     (col+1, row), (col+1, row+1), 
                     (col, row+1), (col-1, row+1)]
        ret = []
        numNeigb = 0
        for pos in positions:
            if (pos[0] < 0 or pos[0] >= len(self.lattice[0]) or
                    pos[1] < 0 or pos[1] >= len(self.lattice)):
                continue
            if self.lattice[pos[1]][pos[0]] == 'noise':
                continue
            else:
                if self.lattice[pos[1]][pos[0]]['sampleCount'] <= self.minSC:
                    self.lattice[pos[1]][pos[0]]['label'] = 'noise'
                else:
                    if not self.lattice[pos[1]][pos[0]]['label']:
                        ret.append(pos)
                    numNeigb += 1
        return ret, numNeigb
    
    def process_node(self, node, curr_cluster):
        q = deque()
        if self.lattice[node[1]][node[0]]['sampleCount'] <= self.minSC:
            self.lattice[node[1]][node[0]]['label'] = 'noise'
            return False
        neighbors, numNeigb = self.get_neighbors(node[0], node[1])
        
        if numNeigb >= self.minNeigb:
            self.lattice[node[1]][node[0]]['clusterId'] = curr_cluster
            self.lattice[node[1]][node[0]]['label'] = 'processed'
            for neighbor in neighbors:
                q.append(neighbor)
                self.lattice[neighbor[1]][neighbor[0]]['label'] = 'unprocessed'
        else:
            return False
        
        while q:
            cur = q.popleft()
            self.lattice[cur[1]][cur[0]]['clusterId'] = curr_cluster
            self.lattice[cur[1]][cur[0]]['label'] = 'processed'
            neighbors, numNeigb = self.get_neighbors(cur[0], cur[1])
            
            if numNeigb >= self.minNeigb:
                for neighbor in neighbors:
                    q.append(neighbor)
                    self.lattice[neighbor[1]][neighbor[0]]['label'] = 'unprocessed'
        return True
    
    def scan(self):
        curr_cluster = 1
        for i in range(self.numRow):
            for j in range(self.numCol):
                if self.lattice[i][j]['label'] == 'noise' or self.lattice[i][j]['label'] == 'processed':
                    continue
                else:
                    node = (j,i)
                    foundCluster = self.process_node(node, curr_cluster)
                    if foundCluster:
                        curr_cluster += 1
                        

""" MOTION PREDICTION """
class motion_prediction:
    def __init__(self, lattice, pdf, present_df, config):
        self.lattice = lattice
        self.pdf = pdf
        self.present_df = present_df
        self.ship_mmsi = present_df["MMSI"].iloc[0]
        
        self.store_param = config['StoreParam']
        self.time_windows = config['TimeWindows']
        self.numCol = config['NumCol']
        self.numRow = config['NumRow']
        self.min_x = config['Min_x']
        self.min_y = config['Min_y']
        self.increment = config['Increment']
        self.threshold = config['Threshold']
        
        self.initial_PTP = None
        self.initial_ELR = None
        self.initial_index_x = None
        self.initial_index_y = None
        self.cur_waterway = None
        self.deviation_ratio = 0
        self.cur_dir = None
        self.cur_heading = None
        self.results = {}
    
    def initialize_variables(self):
        """ Initialize relevant variables from dataset, to be used in motion prediction 
        """
        cur_time = self.present_df['time'].iloc[-1]
        self.initial_index_x, self.initial_index_y = self._coordToIndex(self.present_df['x'].iloc[-1], self.present_df['y'].iloc[-1])
        self.cur_waterway = self.lattice[self.initial_index_y][self.initial_index_x]['clusterId']
        self.cur_heading = self.present_df['Heading'].iloc[-1]
        self.cur_dir = self.encode_direction(self.present_df['Heading'].iloc[-1])
        if self.cur_dir == None:
            _, self.cur_heading = self.compute_traveling_dist(self.time_windows[-1])
            self.cur_dir = self.encode_direction(self.cur_heading)
        self.initial_PTP, self.initial_ELR = self.find_PTP_ELR(self.initial_index_x, self.initial_index_y, self.cur_dir, self.cur_waterway)
        self.deviation_ratio = self.get_deviation_ratio(self.initial_PTP, self.initial_ELR, (self.initial_index_x, self.initial_index_y))
        return True
        
    
    def compute_traveling_dist(self, time_window):
        """ Computes total traveling distance over the past time window (in seconds) 
        """
        cur_time = self.present_df['time'].iloc[-1]
        cur_x = self.present_df['x'].iloc[-1]
        cur_y = self.present_df['y'].iloc[-1]
        vessel_time = self.present_df['time']
        vessel_x = self.present_df['x']
        vessel_y = self.present_df['y']
        
        prev_time = cur_time - time_window
        index = bisect.bisect(vessel_time, prev_time, lo=max(0, len(vessel_time)-time_window), hi=len(vessel_time))
        
        ratio = (prev_time - vessel_time[index-1]) / (vessel_time[index] - vessel_time[index-1])
        prev_x = (vessel_x[index] - vessel_x[index-1]) * ratio + vessel_x[index-1]
        prev_y = (vessel_y[index] - vessel_y[index-1]) * ratio + vessel_y[index-1]
        dist = ((cur_x - prev_x) ** 2 + (cur_y - prev_y) ** 2) ** 0.5
        
        heading = math.degrees(math.atan2(cur_y-prev_y, cur_x-prev_x))
        heading =- 90
        if heading < 0:
            heading *= -1
        elif heading > 0:
            heading = 360 - heading
        
        return dist, heading

    
    def compute_sail_window(self, traveling_dist):
        """ Uses pdf generated by kde to calculate sail window.
            However, we do not need the sail window, thus self.threshold should be None
        """
        if not self.threshold:
            return (traveling_dist - self.increment * 0.5, traveling_dist + self.increment * 1000), None
        prob = 0
        center = self.store_param[2]//2
        pdf_increment = (self.store_param[1] - self.store_param[0]) / (self.store_param[2] - 1)
        i = 0
        prob += self.pdf[center] * pdf_increment
        while prob < self.threshold:
            i += 1
            prob += self.pdf[center+i] * pdf_increment
            prob += self.pdf[center-i] * pdf_increment
        sail_window = (traveling_dist + min(-self.increment*0.5, 0-i*pdf_increment),
                       traveling_dist + max(self.increment*1000, 0+i*pdf_increment))
        
        return sail_window, prob
    
    def find_PTP_ELR(self, index_x, index_y, cur_dir, cur_waterway):
        """ Gets the PTP and ELR of a particular point in lattice
        """
        ELR = deque([(index_x, index_y)])
        if cur_dir in {0,2}:
            for i in range(index_x + 1, len(self.lattice[0])):
                if self.lattice[index_y][i]['clusterId'] == cur_waterway:
                    ELR.append((i, index_y))
                else:
                    break
            for j in reversed(range(index_x)):
                if self.lattice[index_y][j]['clusterId'] == cur_waterway:
                    ELR.appendleft((j, index_y))
                else:
                    break
        else:
            for i in range(index_y + 1, len(self.lattice)):
                if self.lattice[i][index_x]['clusterId'] == cur_waterway:
                    ELR.append((index_x, i))
                else:
                    break
            for j in reversed(range(index_y)):
                if self.lattice[j][index_x]['clusterId'] == cur_waterway:
                    ELR.appendleft((index_x, j))
                else:
                    break
        PTP = ELR[len(ELR)//2]
        return PTP, ELR
    
    def get_neighbor_index(self, index_x, index_y, cur_dir):
        if cur_dir == 0:
            return (index_x, index_y - 1)
        elif cur_dir == 1:
            return (index_x + 1, index_y)
        elif cur_dir == 2:
            return (index_x, index_y + 1)
        else:
            return (index_x - 1, index_y)
        
    def _coordToIndex(self, pos_x, pos_y):
        index_x = math.ceil((pos_x - self.min_x) / self.increment) - 1
        index_y = self.numRow - math.ceil((pos_y - self.min_y) / self.increment)
        if pos_x == self.min_x:
            index_x = 0
        if pos_y == self.min_y:
            index_y = self.numRow - 1
        return index_x, index_y
    
    def check_valid(self, index_x, index_y):
        if 0 <= index_x <= len(self.lattice[0]) - 1 and 0 <= index_y <= len(self.lattice) - 1:
            return True
        else:
            return False
        
    def forecast_trajectory(self):
        """ Main function call to forecast a trajectory
            Returns:
                A list containing dictionaries that store the predicted indexes from t=0 onwards (includes t=0)
        """
        traveling_distances = {}
        sail_windows = [(0,0)] * (self.time_windows[-1]+1)
        temp_results = {}
        start_recursion = 1
        
        cur_x, cur_y = self.present_df['x'].iloc[-1], self.present_df['y'].iloc[-1]
        self.initialize_variables()
        
        if self.cur_waterway == None:
            return None

        if self.cur_dir == 0:
            dist_threshold = self.lattice[self.initial_index_y][self.initial_index_x]['y'] + self.increment - cur_y
        elif self.cur_dir == 1:
            dist_threshold = self.lattice[self.initial_index_y][self.initial_index_x]['x'] + self.increment - cur_x
        elif self.cur_dir == 2:
            dist_threshold = cur_y - self.lattice[self.initial_index_y][self.initial_index_x]['y']
        else:
            dist_threshold = cur_x - self.lattice[self.initial_index_y][self.initial_index_x]['x']
        
        for i in range(1, self.time_windows[-1] +1):
            # Step 1: Compute all traveling distances
            traveling_dist, direction = self.compute_traveling_dist(time_window=i)
            traveling_distances[i] = traveling_dist
            
            if traveling_dist < dist_threshold:
                start_recursion = i+1
                continue

            # Step 2: Compute all sail windows
            if self.pdf:
                sail_window, prob = self.compute_sail_window(traveling_dist=traveling_dist)
            else:
                sail_window, prob = self.compute_sail_window(traveling_dist=traveling_dist)
            sail_windows[i] = sail_window
        
        for j in range(1, start_recursion):
            temp_results[j] = {'PTP': self.initial_PTP, 'ELR': self.initial_ELR, 'deviated_pos': (self.initial_index_x, self.initial_index_y)}

        results = self.forecast_trajectory_dfs(PTP=self.initial_PTP, 
                                               ELR=self.initial_ELR, 
                                               deviated_pos=(self.initial_index_x, self.initial_index_y),
                                               sail_windows=sail_windows, 
                                               accumulated_dist=0,
                                               time=start_recursion,
                                               res=temp_results,
                                               deviation_ratio=self.deviation_ratio,
                                              )
        return results
        
    def forecast_trajectory_dfs(self, PTP, ELR, deviated_pos, sail_windows, accumulated_dist, time, res, deviation_ratio):
        """ Recursive function to get best trajectory
        """
        if time == self.time_windows[-1]+1:
            return [res]
        
        next_path = []
        new_route = True
        for index in ELR:
            neigh_x, neigh_y = self.get_neighbor_index(index[0], index[1], self.cur_dir)
            if not self.check_valid(neigh_x, neigh_y):
                continue
            if new_route and self.lattice[neigh_y][neigh_x]['clusterId'] == self.cur_waterway:
                new_route = False
                new_PTP, new_ELR = self.find_PTP_ELR(neigh_x, neigh_y, self.cur_dir, self.cur_waterway)
                if len(new_ELR) - len(ELR) > 5:
                    new_deviated_pos = self.get_deviated_index(new_PTP, new_ELR, deviated_pos, deviation_ratio, True)
                else:
                    new_deviated_pos = self.get_deviated_index(new_PTP, new_ELR, deviated_pos, deviation_ratio, False)
                temp_deviation_ratio = self.get_deviation_ratio(new_PTP, new_ELR, new_deviated_pos)
                if temp_deviation_ratio == None:
                    new_deviation_ratio = deviation_ratio
                else:
                    if len(new_ELR) - len(ELR) > 5:
                        new_deviation_ratio = 0.95 * temp_deviation_ratio + 0.05 * deviation_ratio
                    else:
                        new_deviation_ratio = 0.05 * temp_deviation_ratio + 0.95 * deviation_ratio
                new_dist = accumulated_dist + ((new_deviated_pos[0] - deviated_pos[0]) ** 2 + (new_deviated_pos[1] - deviated_pos[1]) ** 2) ** 0.5 * self.increment 
                next_path.append((new_PTP, new_ELR, new_deviated_pos, new_dist, new_deviation_ratio))
                
            if self.lattice[neigh_y][neigh_x]['clusterId'] != self.cur_waterway:
                new_route = True
        
        next_path.sort(key=lambda x: x[3])

        results = []
        for path in next_path:
            new_PTP, new_ELR, new_deviated_pos, new_dist, new_deviation_ratio = path
            new_res = res.copy()
            for i in range(time, len(sail_windows)+1):
                if i == len(sail_windows):
                    break
                if sail_windows[i][0] < new_dist < sail_windows[i][1]:
                    new_res[i] = {'PTP': new_PTP, 'ELR': new_ELR, 'deviated_pos': new_deviated_pos}
                else:
                    break
            results += self.forecast_trajectory_dfs(new_PTP, new_ELR, new_deviated_pos, sail_windows, new_dist, i, new_res, new_deviation_ratio)
        
        return results
    
    
    def encode_direction(self, heading):
        if not heading:
            return heading
        else:
            if heading <= 45 or heading > 315:
                return 0
            elif 45 < heading <= 135:
                return 1
            elif 135 < heading <= 225:
                return 2
            else:
                return 3
        
    
    def get_deviation_ratio(self, PTP, ELR, deviated_pos):
        """ Deviation ratio is the perpendicular proportion of a position from its PTP
        """
        try:
            if self.cur_dir in {0,2}:
                if PTP[0] > deviated_pos[0]:
                    ratio = -(PTP[0] - deviated_pos[0]) / (PTP[0] - ELR[0][0])
                elif PTP[0] < deviated_pos[0]:
                    ratio = (PTP[0] - deviated_pos[0]) / (PTP[0] - ELR[-1][0])
                else:
                    ratio = 0
            else:
                if PTP[1] > deviated_pos[1]:
                    ratio = -(PTP[1] - deviated_pos[1]) / (PTP[1] - ELR[0][1])
                elif PTP[1] < deviated_pos[1]:
                    ratio = (PTP[1] - deviated_pos[1]) / (PTP[1] - ELR[-1][1])
                else:
                    ratio = 0
            return ratio
        except:
            return None
    
    def get_deviated_index(self, PTP, ELR, prev_deviated_pos, deviation_ratio, largeIncrease):
        """ Function applies deviation ratio onto a PTP to return the deviated indexes
            Here, we handle the deviation to ensure the changing position is not too large
        """
        if self.cur_dir in {0,2}:
            pred_y = int(PTP[1])
            if deviation_ratio > 0:
                pred_x = int(math.floor(PTP[0] + deviation_ratio * (ELR[-1][0] - PTP[0])))
            elif deviation_ratio < 0:
                pred_x = int(math.ceil(PTP[0] + deviation_ratio * (PTP[0] - ELR[0][0])))
            else:
                pred_x = int(PTP[0])
            while pred_x - prev_deviated_pos[0] > 1 and self.lattice[pred_y][pred_x-1]['clusterId'] == self.cur_waterway:
                pred_x -= 1
            while prev_deviated_pos[0] - pred_x > 1 and self.lattice[pred_y][pred_x+1]['clusterId'] == self.cur_waterway:
                pred_x += 1
            if largeIncrease:
                if self.cur_heading < 20 or self.cur_heading > 340 or 160 < self.cur_heading < 200:
                    while pred_x - prev_deviated_pos[0] > 0 and self.lattice[pred_y][pred_x-1]['clusterId'] == self.cur_waterway:
                        pred_x -= 1
                    while prev_deviated_pos[0] - pred_x > 0 and self.lattice[pred_y][pred_x+1]['clusterId'] == self.cur_waterway:
                        pred_x += 1
                elif 20 <= self.cur_heading <= 45 or 135 <= self.cur_heading <= 160:
                    while pred_x - prev_deviated_pos[0] < 1 and self.lattice[pred_y][pred_x+1]['clusterId'] == self.cur_waterway:
                        pred_x += 1
                elif self.cur_heading >= 315 or 200 <= self.cur_heading <= 225:
                    while prev_deviated_pos[0] - pred_x < 1 and self.lattice[pred_y][pred_x-1]['clusterId'] == self.cur_waterway:
                        pred_x -= 1
                    
        else:
            pred_x = int(PTP[0])
            if deviation_ratio > 0:
                pred_y = int(math.floor(PTP[1] + deviation_ratio * (ELR[-1][1] - PTP[1])))
            elif deviation_ratio < 0:
                pred_y = int(math.ceil(PTP[1] + deviation_ratio * (PTP[1] - ELR[0][1])))
            else:
                pred_y = int(PTP[1])
            while pred_y - prev_deviated_pos[1] > 1 and self.lattice[pred_y-1][pred_x]['clusterId'] == self.cur_waterway:
                pred_y -= 1
            while prev_deviated_pos[1] - pred_y > 1 and self.lattice[pred_y+1][pred_x]['clusterId'] == self.cur_waterway:
                pred_y +=1
            if largeIncrease:
                if 70 < self.cur_heading < 110 or 250 < self.cur_heading < 290:
                    while pred_y - prev_deviated_pos[1] > 0 and self.lattice[pred_y-1][pred_x]['clusterId'] == self.cur_waterway:
                        pred_y -= 1
                    while prev_deviated_pos[1] - pred_y > 0 and self.lattice[pred_y+1][pred_x]['clusterId'] == self.cur_waterway:
                        pred_y += 1
                elif 290 <= self.cur_heading <= 315 or 45 <= self.cur_heading <= 70:
                    while prev_deviated_pos[1] - pred_y < 1 and self.lattice[pred_y-1][pred_x]['clusterId'] == self.cur_waterway:
                        pred_y -= 1
                elif 225 <= self.cur_heading <= 250 or 110 <= self.cur_heading <= 135:
                    while pred_y - prev_deviated_pos[1] < 1 and self.lattice[pred_y+1][pred_x]['clusterId'] == self.cur_waterway:
                        pred_y += 1
                    
        return (pred_x, pred_y)
    
    
    def summarize_traj(self, results):
        """ Function filters results to the most probable one and returns it
        """
        if results == None or len(results) <= 0:
            return None
        res = results[0]
        ret = {}
        for key in res.keys():
            ret[key] = res[key]['deviated_pos']
        return ret
    
    
    def smoothen_traj(self, results):
        """ Function takes the results and produce a trajectory of points
        """
        if results == None or len(results) <= 0:
            return None
        
        from collections import Counter  
        counter = Counter()
        sequence = []
        for key in results.keys():
            counter[results[key]] += 1
            if counter[results[key]] == 1:
                sequence.append(results[key])
        
        cur_x = self.present_df['x'].iloc[-1]
        cur_y = self.present_df['y'].iloc[-1]
        
        trajectory = []
        if len(sequence) <= 1:
            return [(cur_x, cur_y)] * len(results)
        
        # Handle edge case (first index)
        endPoint = self.getEndPoint(sequence[0], sequence[1])
        self.interpolatePoints((cur_x, cur_y), endPoint, counter[sequence[0]], trajectory)
        startPoint = endPoint
        # Handle all cases in between
        for i in range(1, len(sequence)-1):
            endPoint = self.getEndPoint(sequence[i], sequence[i+1])
            self.interpolatePoints(startPoint, endPoint, counter[sequence[i]], trajectory)
            startPoint = endPoint
        # Handle edge case (last index)
        sequence[-1]
        endPoint = (self.lattice[sequence[-1][1]][sequence[-1][0]]['x'], self.lattice[sequence[-1][1]][sequence[-1][0]]['y'])
        self.interpolatePoints(startPoint, endPoint, counter[sequence[-1]], trajectory)
        assert len(trajectory) == len(results)
        return trajectory
    
    
    def interpolatePoints(self, startPoint, endPoint, num, traj):
        """ Helper function to interpolate between points to smoothen trajectory
        """
        diff_x = endPoint[0] - startPoint[0]
        diff_y = endPoint[1] - startPoint[1]
        for i in range(1, num+1):
            traj.append((startPoint[0] + i/num*diff_x, startPoint[1] + i/num*diff_y))
        return True
    
    
    def getEndPoint(self, curIndexes, nextIndexes):
        """ Obtain the end point of a lattice space (center, upper/left or lower/right)
        """
        if self.cur_dir in {0,2}:
            if self.cur_dir == 0:
                yCoord = self.lattice[curIndexes[1]][curIndexes[0]]['y'] + self.increment / 2
            elif self.cur_dir == 2:
                yCoord = self.lattice[curIndexes[1]][curIndexes[0]]['y'] - self.increment / 2
            if curIndexes[0] - nextIndexes[0] < 0:
                xCoord = self.lattice[curIndexes[1]][curIndexes[0]]['x'] + self.increment / 2
            elif curIndexes[0] - nextIndexes[0] > 0:
                xCoord = self.lattice[curIndexes[1]][curIndexes[0]]['x'] - self.increment / 2
            else:
                xCoord = self.lattice[curIndexes[1]][curIndexes[0]]['x']
        
        else:
            if self.cur_dir == 1:
                xCoord = self.lattice[curIndexes[1]][curIndexes[0]]['x'] + self.increment / 2
            elif self.cur_dir == 3:
                xCoord = self.lattice[curIndexes[1]][curIndexes[0]]['x'] - self.increment / 2
            if curIndexes[1] - nextIndexes[1] < 0:
                yCoord = self.lattice[curIndexes[1]][curIndexes[0]]['y'] - self.increment / 2
            elif curIndexes[1] - nextIndexes[1] > 0:
                yCoord = self.lattice[curIndexes[1]][curIndexes[0]]['y'] + self.increment / 2
            else:
                yCoord = self.lattice[curIndexes[1]][curIndexes[0]]['y']
            
        return (xCoord, yCoord)
        
        
    def reset_all(self):
        """ Function to reset all preset variables
        """
        for i in range(self.numRow):
            for j in range(self.numCol):
                self.lattice[i][j]['code'] = None
        self.results = {}
        self.initial_PTP = None
        self.initial_index_x = None
        self.initial_index_y = None
        self.cur_dir = None
        self.deviation_ratio = 0
        return True

    
""" Smoothing trajectory algorithm """
# Smooth each point based on a loss function and tolerance
from copy import deepcopy
def smooth(path, weight_data=0.5, weight_smooth=0.1, tolerance=0.000001):
    new = deepcopy(path)
    dims = len(path[0])
    change = tolerance
    
    while change >= tolerance:
        change = 0.0
        for i in range(1, len(new)-1):
            for j in range(dims):
                x_i = path[i][j]
                y_i, y_prev, y_next = new[i][j], new[i-1][j], new[i+1][j]
                
                y_i_saved = y_i
                y_i += weight_data * (x_i - y_i) + weight_smooth * (y_next + y_prev - (2 * y_i))
                new[i][j] = y_i
                
                change += abs(y_i - y_i_saved)
        return new


# bspline
import scipy.interpolate as interpolate
def bsplineSmooth(trajectory):
    temp = trajectory.copy()
    temp.sort(key=lambda x: x[0], reverse=False)
    x, y = np.array(temp)[:,0], np.array(temp)[:,1]
    
    t, c, k = interpolate.splrep(x, y, k=3, s=0)
    xin, xmax = x.min(), x.max()
    xx = np.linspace(xmin, xmax, 100)
    spline = interpolate.BSpline(t, c, k, extrapolate=False)
    
    return spline, xx

In [ ]:
""" Example Usage """

# Set Config file
FilterMap = [4090000,4070000,5130000,5110000]
TimeWindows = [i for i in range(1, 61)]
config = {'ScaleDown': 1,
          'VesselType': None,
          'FilterMap': FilterMap,
          'NumCol': 1000,
          'NumRow': 1000,
          'Increment': 20,
          'Min_x': FilterMap[1],
          'Min_y': FilterMap[3],
          'MinSC': 2,
          'MinNeigb': 4,
          'StoreParam': [-4, 4, 101],
          'TimeWindows': TimeWindows,
          'Threshold': None,
          }

# Read preprocessed data
df_processed = pd.read_csv('./ProcessedData_engined_wutm.csv')
df_processed = df_processed[['MMSI', 'x', 'y', 'time', 'VesselType', 'Heading']]
print('Preprocessed data read')

# DBSCAN to build lattice
start = time.time()
DBSCAN_object = DBSCAN(df_processed, config)
DBSCAN_object.scan()
lattice = DBSCAN_object.lattice
end = time.time()
print('Runtime for DBSCAN is: {}'.format(end-start))

# Extract dataset
df_test_original = df_processed.copy()
df_test_original.sort_values(by=['MMSI', 'time'], inplace=True, ascending=True)
df_test_original = df_test_original.reset_index()
df_test_original = df_test_original[['MMSI', 'x', 'y', 'time', 'VesselType', 'Heading']]
df_test_original = df_test_original.groupby('MMSI').filter(lambda x: len(x) > 10000)
shipName = df_test_original['MMSI'].unique()[1]
df_test_original = df_test_original[df_test_original['MMSI'] == shipName]
print('Data has been filtered to only 1 ship name')

# Predict motion
start = time.time()
MotionPrediction = motion_prediction(lattice, None, df_test, config)
all_traj = testMotionPrediction.forecast_trajectory()
predicted_index = testMotionPrediction.summarize_traj(all_traj)
predicted_traj = testMotionPrediction.smoothen_traj(predicted_index)
for i in range(len(predicted_traj)):
    predicted_traj[i] = list(predicted_traj[i])
new_traj = smooth(predicted_traj)
end = time.time()
print('Runtime for one round of motion prediction is: {}'.format(end-start))

# new_traj is the predicted trajectory for use